In [1]:
import pandas as pd
import numpy as np
import os 
import time
from datetime import datetime, date, timedelta
%config IPCompleter.greedy=True
%config Completer.use_jedi = False

In [2]:
# what files to work
# All_spec or history (yes)
while True:
    final_file_type = input("Do you want the All_spec file or history file? (All_spec or history)  ")
    if final_file_type == "All_spec" or final_file_type == "history":
        break
# All the data or just the current
get_current_specs = input("Do you want to get only the Current specs? (yes or no)  ")

Do you want the All_spec file or history file? (All_spec or history)  All_spec
Do you want to get only the Current specs? (yes or no)  no


# First Steps:
  * Uploading the data
  * tyding up
  * choosing the whole data or only Current data

In [3]:
#####################
# Uploading the data#
#####################
#
data_source_path = "sources/allsap.txt"
all_sap = pd.read_csv(data_source_path, sep='\t')
# Uploading Structure data ( was done manually)
structures = pd.read_csv('sources/structures.csv')
# getting the searcher DF 
if final_file_type == "history":
    searcher = pd.read_csv("sources/AllAttributes.csv")
elif final_file_type == "All_spec":
    searcher = pd.read_csv("sources/AllAttributes_Local.csv")
    
# getting the operations df
operations = pd.read_csv('sources/Operations.csv')
#
################################
# Dropping the non useful data #
################################
#
#list of columns to be dropped
to_drop = ['Unnamed: 0','Unnamed: 4','Unnamed: 5','Unnamed: 6','Unnamed: 7','Unnamed: 9','Unnamed: 11','Unnamed: 13','Unnamed: 14','MANDT','WERKS']
all_sap.drop(to_drop, inplace=True, axis=1)
all_sap['UDAT'] = pd.to_datetime(all_sap['UDAT'],dayfirst=True)
all_sap = all_sap.set_index('UDAT') 
#
###########################################
# if you want to Get only the current spec#
###########################################
if get_current_specs == "yes":
    rp_docs = all_sap[(all_sap["VALUE"]=='RP') & 
                      (~all_sap["DOKNR"].str.startswith('YA')) & 
                      (all_sap["CHARACT"]== all_sap["CLASS"].str[4:8] + "_STATUS")]['DOKNR']
    rp_docs = set (rp_docs)
    exclusion_list = ['SAH0561AUEG02','SAH0566AGEG03','SAH0569AAEG0D','SAH0570AFEG0M','SAH0570AGEG0M','SAH0570AIEG0A','SAH0571AJEG0B','SAH0571AKEG0A','SAH0618AEEG01','SAH0645A2EG08']
    rp_docs = rp_docs - set(exclusion_list)
    
    all_sap = all_sap[all_sap["DOKNR"].isin(rp_docs)]
                      
all_sap.to_csv('good_master_data.csv')

  * Getting the missing AHHA structures

In [4]:
# find the structures missing from my file to add to structures file
strcts_eg = all_sap[(all_sap["CHARACT"]=="AHAA_STRUCT") & \
                    (all_sap["MATNR"].str.contains("SAH"))]['VALUE'].drop_duplicates()
my_current_struct = structures[structures['CLASS']=="02K6AHAA0"]['STRUCT'].drop_duplicates()
struct_to_add = list(set(strcts_eg)-set(my_current_struct))
print ("AHHA Structures need to be added: ", struct_to_add)

# find the classes missing from my file to add to the structures file
classes_eg = all_sap['CLASS'].drop_duplicates()
my_current_classes = structures['CLASS'].drop_duplicates()
classes_to_add = list(set(classes_eg)-set(my_current_classes))
print ("Classes need to be added to structures file: ", classes_to_add)


# find the classes missing from my file to add to the ALL ATTRIBUTES file
my_current_classes_in_AllAttributrs = searcher['COMPONENT_CLASS'].drop_duplicates()
classes_to_add_to_AllAttributes = list(set(classes_eg)-set(my_current_classes_in_AllAttributrs))
classes_to_add_to_AllAttributes = list(set(classes_to_add_to_AllAttributes) -  set(['0201IMAAX', '02K6LEAA0', '02K6CXAA0', '02K6ZPAC0', '02K6CRAA0', '02K6CNAD0', '02K6ZPAF0', '0203G1MEX', '02K6ZPAA0', '02K6CEBD0', '02K6CNAS0', '02K6BRAR0', '02K6CNAV0']))
print ("Classes need to be added to AllAttributes file: ", classes_to_add_to_AllAttributes)


f1= input('Do you want to continue')
if f1 == "no":
    raise SystemExit("Stop right there!")

AHHA Structures need to be added:  []
Classes need to be added to structures file:  []
Classes need to be added to AllAttributes file:  []
Do you want to continueyes


# 1 Getting all Connections

in this file we have only one goal
to get a DF tah contains all connections 
* SAH, SAH_DOC   SAW, SAW_DOC   


In [5]:
#####################################
#1.1getting SAW and SAH relationship#
#####################################
saw_sah =all_sap[(all_sap['DOKNR'].str.contains('SAW')                    ) & \
                      (   (all_sap['CHARACT']=='AWAA_COMPONENT_AH') | 
                          (all_sap['CHARACT']=='C_AWAA_EG_CODICE_CORTO') | 
                          (all_sap['CHARACT']=='AWAA_RELEASE_DATE')            )
                                         ].copy(deep=True)

#droping columns 
saw_sah.drop(['ZCOUNT','CLASS','COMPONENT'], inplace=True, axis=1)
# pivoting to create new columns for each CHARCT
saw_sah = pd.pivot_table(saw_sah, 
                              index=['UDAT','MATNR','DOKNR'], 
                              columns='CHARACT',
                              values='VALUE',
                              aggfunc=lambda x: x)
saw_sah.columns.name = None               #remove groups in columns
saw_sah = saw_sah.reset_index()                #index to columns

#renaming Some
saw_sah.rename(columns= {'MATNR':'SAW',
                              'DOKNR':'SAW_DOC',
                              'AWAA_COMPONENT_AH':'SAH', 
                              'AWAA_RELEASE_DATE':'SAW_RELS_DAT',
                              'C_AWAA_EG_CODICE_CORTO':'SAW_ShortCode'}, inplace=True)
saw_sah.dropna(subset=['SAH'], inplace = True)
# correcting the time format 
saw_sah['SAW_RELS_DAT'] = pd.to_datetime(saw_sah['SAW_RELS_DAT'], format='%d/%m/%Y')
saw_sah.set_index('UDAT', inplace= True)



# #######################################################
# #1.2getting SAH and SAH_DOC and Structure relationship#
# #######################################################

#getting the sah document structure.#

sah_sahDoc = all_sap[(all_sap['MATNR'].str.contains('SAH')) &
                             (     (all_sap['CHARACT']=='AHAA_STRUCT') |
                                   (all_sap['CHARACT']=='AHAA_RELEASE_DATE') | 
                                   (all_sap['CHARACT']=='AHAA_RELEASE_DATE') |
                                   (all_sap['CHARACT']=='C_AHAA_EG_CODICE_CORTO')  
                             )].copy(deep= True)
#droping non needed columns
sah_sahDoc.drop(['ZCOUNT','CLASS','COMPONENT'], inplace=True, axis=1)
# pivoting to create new columns for each CHARCT
sah_sahDoc = pd.pivot_table(sah_sahDoc, 
                                      index=['UDAT','MATNR','DOKNR'], 
                                      columns='CHARACT',
                                      values='VALUE',
                                      aggfunc=lambda x: x)
sah_sahDoc.columns.name = None               #remove groups in columns
sah_sahDoc = sah_sahDoc.reset_index()                #index to columns
#renaming Some
sah_sahDoc.rename(columns= {'DOKNR':'SAH_DOC', 
                                     'MATNR':'SAH',
                                     'AHAA_RELEASE_DATE':'SAH_RELS_DAT', 
                                     'C_AHAA_EG_CODICE_CORTO':'SAH_ShortCode'}, inplace=True)
# # correcting the time format 
sah_sahDoc['SAH_RELS_DAT'] = pd.to_datetime(sah_sahDoc['SAH_RELS_DAT'], format='%d/%m/%Y')
sah_sahDoc.set_index('UDAT', inplace= True)

# ################################################
# # 1.3 Getting SAH_DOC, SAW_DOC and their dates #
# ################################################
sahs = sah_sahDoc.copy(deep = True)
sahs.reset_index( inplace= True)
sahs.rename (columns = {'UDAT':'SAH_DAT'}, inplace= True)
sahs.drop(columns=['AHAA_STRUCT'], inplace = True)

saws = saw_sah.copy(deep = True)
saws.reset_index( inplace= True)
saws.rename (columns = {'UDAT':'SAW_DAT'}, inplace= True)
# saws.drop(columns=['SAH','AHAA_STRUCT'], inplace = True)
asma = pd.merge(saws, sahs, left_on=['SAH','SAW_ShortCode'], right_on=['SAH','SAH_ShortCode'], how = 'inner')
asma.drop_duplicates('SAH_DOC', keep='last', inplace = True )
asma.drop(columns=['SAW_ShortCode','SAH_ShortCode'], axis=1 , inplace= True)



###############################################
# 1.4 Getting the Date of SAW_DOC termination #
###############################################


# getting the production status for each saw doc
unique_SAW_DOC = pd.DataFrame(saw_sah['SAW_DOC'])
unique_SAW_DOC['Indu_Status_helper'] = "AWAA_STATUS"
SAW_DOC_STATE = pd.merge(unique_SAW_DOC, all_sap, 
                         left_on=['SAW_DOC','Indu_Status_helper'], 
                         right_on = ['DOKNR','CHARACT'], how = 'left')
SAW_DOC_STATE.drop(['Indu_Status_helper','MATNR','DOKNR','ZCOUNT','CLASS','CHARACT','COMPONENT'], 
                   inplace = True, axis=1)

# getting the release date for all saw doc
SAW_DOC_STATE_RelsDate = pd.merge(SAW_DOC_STATE,saw_sah[['SAW_DOC','SAW_RELS_DAT']], on= 'SAW_DOC')



#Changing the date of only RP saw doc to be the current date
# Get file's Last modification time stamp only in terms of seconds since epoch 
modTimesinceEpoc = os.path.getctime(data_source_path)
# Convert seconds since epoch to readable timestamp
# extractionTime = time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(modTimesinceEpoc))
extractionTime = time.strftime('%d-%m-%y %H:%M:%S', time.localtime(modTimesinceEpoc))
# print("Last Modified Time : ", modificationTime )

SAW_DOC_STATE_RelsDate.loc[SAW_DOC_STATE_RelsDate['VALUE'] == 'RP', 'SAW_RELS_DAT'] = extractionTime
# SAW_DOC_STATE_RelsDate['SAW_RELS_DAT'] = SAW_DOC_STATE_RelsDate['SAW_RELS_DAT'].dt.strftime('%Y-%m-%d')
SAW_DOC_STATE_RelsDate.set_index('SAW_DOC', inplace = True)




# doing the same of SAH (changing the dates of RP SAH to be as extraction time)
# the list of SAH that are currently RP
sahDoc_RP = all_sap[(all_sap['MATNR'].str.contains('SAH')) &((all_sap['CHARACT']=='AHAA_STATUS'))&((all_sap['VALUE']=='RP'))
                   ]['DOKNR'].copy(deep= True)
asma.loc[asma['SAH_DOC'].isin(sahDoc_RP),'SAH_DAT'] = extractionTime
asma['SAH_DAT'] = asma['SAH_DAT'].astype('datetime64')

# 2 Getting all the SC levels

in this file we have only one goal
to get a DF tah contains 
* SAH_DOC
* All_COMPONENTS

In [6]:
#############################################################
# 2.1 getting date, SAH and SAH_Components (the first level)#
#############################################################
## 2.1.1 adding the data from SAH_Doc to know the Class of each Semicomponent and the SC themselves
sah_sahDoc_classes = pd.merge(sah_sahDoc, structures, left_on='AHAA_STRUCT', right_on='STRUCT',how = 'left')
sah_sahDoc_classes.drop(['SAH','CLASS','STRUCT','CHARCT_toGetSublen','CHARCT_toGetSubWeight'], inplace= True, axis =1 )
#might need something from above to try and if no keep it

## 2.1.2 getting the Semi-Component documents using CHARCT_toGetSubDoc 
sah_sahDoc_classes_compDoc = pd.merge(sah_sahDoc_classes,all_sap, left_on = ['SAH_DOC','CHARCT_toGetSubDoc'],
                                     right_on = ['DOKNR','CHARACT'])
##dropping 
sah_sahDoc_classes_compDoc.drop(['CLASS','MATNR','DOKNR','ZCOUNT','COMPONENT','AHAA_STRUCT','SUB_EF','CHARCT_toGetSubDoc','CHARACT','SAH_ShortCode'], axis=1 , inplace = True)
sah_sahDoc_classes_compDoc.rename(columns={'VALUE':'COMPONENT_DOC','SUB_CLASSES':'COMPONENT_CLASS'}, inplace = True)

#######################################################
# 2.2 to get the sub sub components (the second level)#
#######################################################

suba = sah_sahDoc_classes_compDoc.copy(deep = True)
subs = sah_sahDoc_classes_compDoc.copy(deep = True)
for i in range(0, 10):
    suba = pd.merge(suba, 
                   structures[['CLASS','CHARCT_toGetSubDoc','SUB_CLASSES']], 
                   left_on=['COMPONENT_CLASS'], right_on=['CLASS'], how='left')
    suba.drop(['COMPONENT_CLASS','CLASS'], inplace= True, axis=1)

    # 2.2.2 getting the S-C documents using CHARCT_toGetSubDoc 
    suba = pd.merge(suba, all_sap[['DOKNR','CHARACT','VALUE']], 
                       left_on=['COMPONENT_DOC','CHARCT_toGetSubDoc'], right_on= ['DOKNR','CHARACT'], how='left')
    ## sub_sub = sub_sub.dropna()
    suba.drop(columns={'CHARCT_toGetSubDoc','DOKNR','CHARACT','COMPONENT_DOC'}, inplace= True)
    suba.rename(columns={'SUB_CLASSES':'COMPONENT_CLASS','VALUE':'COMPONENT_DOC'}, inplace = True)
    suba.dropna(subset = ['COMPONENT_DOC'], inplace = True)
    suba.drop_duplicates(subset=["SAH_DOC","COMPONENT_NAME","COMPONENT_CLASS","COMPONENT_DOC"],inplace = True)
    
    
    subs = pd.concat(
    [subs,suba],
    axis=0,
    join="outer",
    ignore_index=False,
    keys=None,
    levels=None,
    names=None,
    verify_integrity=False,
    copy=True)

subs.drop_duplicates(inplace = True)

######################
# 2.3 adding SAW lev #
######################
sawDoc_as_component = asma[['SAH_DOC','SAW_DOC','SAH_RELS_DAT']].copy(deep= True)
# renaming columns to mach the ones of SAH_Components df
sawDoc_as_component.rename(columns={'SAW_DOC':'COMPONENT_DOC'}, inplace = True)
sawDoc_as_component['COMPONENT_CLASS'] = '02K6AWAA0'
sawDoc_as_component['COMPONENT_NAME']= "IP"

###########################################
# 2.4 adding SAH_DOC in the somponent col # 
###########################################

SahDoc_as_component = sah_sahDoc_classes_compDoc.drop_duplicates('SAH_DOC').copy(deep = True )
SahDoc_as_component['COMPONENT_DOC']=SahDoc_as_component['SAH_DOC']
SahDoc_as_component['COMPONENT_CLASS']='02K6AHAA0'
SahDoc_as_component['COMPONENT_NAME']= "Central"


#####################
# 2.5 Concatinating # 
#####################

# 2.5.1 The concatination
sahDoc_Comp = pd.concat(
    [sah_sahDoc_classes_compDoc,subs,sawDoc_as_component,SahDoc_as_component],
    axis=0,
    join="outer",
    ignore_index=False,
    keys=None,
    levels=None,
    names=None,
    verify_integrity=False,
    copy=True)

# 2.5.2 to add the UDAT (not sure if needed)
s = sah_sahDoc.copy(deep=True)
s.reset_index(inplace=True)
sahDoc_Comp = pd.merge(sahDoc_Comp, s[['UDAT','SAH_DOC']], on= 'SAH_DOC', how='left')
sahDoc_Comp = sahDoc_Comp.copy(deep = True)

# 3 The Merge

Result DF: 
  * SAH_DOC
  * Col_index
  * VALUE
  * col_name: might not need


In [7]:
#########################################################################################
# 3.1 Merging all_sap with searcher (the list of attrubutes and their place in all_spec)#
#########################################################################################
all_sap1= all_sap.copy(deep = True)
all_sap1.drop(columns=['ZCOUNT','COMPONENT','MATNR'], inplace= True,axis=1)
#renaming so we have only one column after merging
all_sap1.rename(columns={'CLASS':'COMPONENT_CLASS','DOKNR':'COMPONENT_DOC','CHARACT':'TECH_NAME'} , inplace = True)
# merging all_sap with searcher to get results DF 
data_id  = pd.merge(sahDoc_Comp,searcher, on=['COMPONENT_CLASS','COMPONENT_NAME'], how='right')


#############################################################
# 3.2 merging the data_id with sahDoc_Comp to get full_data #
#############################################################
#we need to get
full_data = pd.merge(data_id, all_sap1, left_on=['COMPONENT_DOC','COMPONENT_CLASS','TECH_NAME'], 
                   right_on=['COMPONENT_DOC', 'COMPONENT_CLASS','TECH_NAME'], how = 'right')
full_data.dropna(inplace= True) ##############################################################################?

full_data.set_index('UDAT', inplace= True)
full_data.drop_duplicates(inplace= True)

# 4) The arranging

In [8]:
if get_current_specs == 'yes':
    # straight to the pivot
    resultsLocal = full_data.copy(deep =True)
    resultsLocal.reset_index(inplace = True)
    resultsLocal.sort_values(['SAH_DOC','col_index'], inplace= True)
    resultsLocal = resultsLocal[resultsLocal['col_index'] != -1]   
    
    hey = pd.pivot_table(resultsLocal,index=['SAH_DOC'],columns='col_index',values='VALUE',
                         aggfunc='first')    

    
else:

    #######################
    # 4.1 Getting mondays #
    #######################

    # Getting all Mondays in years ..
    if get_current_specs == 'yes':
        my_start = date.today() - timedelta(7)
    else:
        my_start = '2008-01-01'
    my_end = pd.to_datetime("today")
    mondays = pd.date_range(start = my_start ,end = my_end , freq = "W-MON")

    # putting them in a new df
    one_column_df = pd.DataFrame(mondays)
    one_column_df.rename(columns = {0:'Date'}, inplace = True)



    ################
    # 4.2 the loop #
    ################
    weeks_sawDoc_sahDoc_list = []

    # might need to convert back to SAW rather than SAW_DOC or use the SAW_DAT for both start and end
    for sawDoc in saw_sah['SAW_DOC'].unique():
        all_version_df = asma[asma['SAW_DOC']== sawDoc]
        #getting start and end date of the SAW document
        t0a = all_version_df['SAW_RELS_DAT'].min()
        t0b = all_version_df['SAW_DAT'].min()
        t0c = all_version_df['SAH_RELS_DAT'].min()
        t0d = all_version_df['SAH_DAT'].min()
        saw_start = min([t0a, t0b, t0c, t0d])


        saw_last = SAW_DOC_STATE_RelsDate.loc[sawDoc,'SAW_RELS_DAT']

        #greater than the start date and smaller than the end date
        mask = (one_column_df['Date'] > saw_start) & (one_column_df['Date'] <= saw_last)

        timeFrameDf = one_column_df.loc[mask]

        # merging and filling the weeks
        timeWithSah = pd.merge_asof (timeFrameDf, 
                                     all_version_df.sort_values('SAH_DAT'), 
                                     left_on= ['Date'], right_on=['SAH_DAT'], direction='forward')
        timeWithSah= timeWithSah[['Date','SAW_DOC','SAH_DOC']]

        #add to master list
        weeks_sawDoc_sahDoc_list.append(timeWithSah)


    weeks_sawDoc_sahDoc_df = pd.concat(weeks_sawDoc_sahDoc_list)


    #################################################################
    # 4.3 Merging the data with required dates (a sample each week) # 
    #################################################################

    weeks_sawDoc_sahDoc_df.dropna( inplace= True)
    resultsWweeks = pd.merge(weeks_sawDoc_sahDoc_df,full_data, on=['SAH_DOC'], how ='left' )
    resultsWweeks.sort_values(['Date','SAW_DOC','SAH_DOC','col_index'], inplace= True)
    resultsWweeks = resultsWweeks[resultsWweeks['col_index'] != -1]

    ####################
    # 4.4 Re-arranging #
    ####################
    hey = pd.pivot_table(resultsWweeks,index=['Date','SAW_DOC','SAH_DOC'],columns='col_index',values='VALUE',
                   aggfunc='first')



In [9]:
####################
# 4.5 Final tuning #
####################

#finding the missing columns
# all columns there is
all_cols = searcher[['col_index']].copy(deep = True)
all_cols.drop_duplicates(inplace = True)
all_cols = all_cols[all_cols['col_index'] != -1]
all_cols_list = list(all_cols['col_index'])
# the columns present in the final DF
current_list = list(hey.columns)
# missing cols
missing_cols = list(set(all_cols_list) - set(current_list))

hey[missing_cols]= ""
salma = hey.reindex(sorted(hey.columns), axis=1)


# firist drop the index 
salma.reset_index (inplace = True)

if final_file_type == "history":
    # getting only the results I need 2018 onwards
#     salma = salma[(salma['Date']>='2017-12-31')]
    salma = salma[(salma['Date']>='2021-12-31')]

# 5) Manual shit

In [10]:
if final_file_type == "history":
    # Plant col
    salma.loc[:,1] = 'M001'
    # Date col
    salma.loc[:,2] = salma.loc[:,'Date']
    # Week col 
    salma.loc[:,3] = salma.loc[:,'Date'].dt.isocalendar().week
    # month name col
    salma.loc[:,4] = salma.loc[:,'Date'].dt.month_name()
    # year name
    salma.loc[:,5] = salma.loc[:,'Date'].dt.year
    # IPCODE/Description
    salma.loc[:,7] = salma.loc[:,9].str.cat(salma.loc[:,11], sep ="/")





    # Converting commas to dots

    cols_to_convert = list(set(operations[operations['is_numer']=='Yes']['col_index']))
    # 1 replace all "."" with nothing (regex false or true dont know)
    salma.loc[:, (cols_to_convert)] = salma.loc[:, (cols_to_convert)].apply(lambda src: src.str.replace('.', '', regex = True))
    # 2 replace all the "," with "." (regex false or true dont know)
    salma.loc[:, (cols_to_convert)] = salma.loc[:, (cols_to_convert)].apply(lambda src: src.str.replace(',', '.', regex = True))
    # 3 do the operation
    salma.loc[:, (cols_to_convert)] = salma.loc[:, (cols_to_convert)].astype(float, errors = 'ignore')
    # salma.loc[:,(cols_to_convert)]



    #Making the operations
    # dividing operations
    # the rows where the operation is required only
    to_operate_onDF = operations[(operations['needs_work']=="YES") &(operations['Operation_type']=="/")].copy(deep = True)
    # The list of columns that needs operations
    cols_to_operate = list(set(to_operate_onDF['col_index']))
    # Setting the col index as index
    to_operate_onDF.set_index('col_index', inplace = True)
    # to convers the columns needing operations to floats in the big df
    salma.loc[:, (cols_to_operate)] = salma.loc[:, (cols_to_operate)].astype(float, errors = 'ignore')
    # Converting the column of factor to float
    to_operate_onDF.loc[:,'factor'] = to_operate_onDF.loc[:,'factor'].astype(float, errors = 'ignore')

    #The operation
    salma.loc[:, cols_to_operate] = salma.loc[:, cols_to_operate]/ to_operate_onDF.T.loc['factor',cols_to_operate]



    # total SideWall Volume


    #droping the columns not needed
    salma.drop( columns = ['Date','SAW_DOC','SAH_DOC'], inplace = True)
    #renaming the col index

    col_names_index_df = searcher[['col_index','COMPONENT_ATTRIB']]
    col_names_index_df = col_names_index_df.copy(deep = True)
    col_names_index_df.drop_duplicates(subset=("col_index"),inplace = True)
    salma.rename(columns=dict(zip(col_names_index_df["col_index"], col_names_index_df["COMPONENT_ATTRIB"])), inplace = True)
    #setting plant as index instead of col_index
    salma.set_index( 'PLANT' , inplace = True)
    salma.to_excel('All_spec.xlsx')

    
    
elif final_file_type == "All_spec":
    
    #droping the columns not needed
    salma.drop( columns = ['SAH_DOC'], inplace = True)
    
    #renaming the col index
    col_names_index_df = searcher[['col_index','COMPONENT_ATTRIB']]
    col_names_index_df = col_names_index_df.copy(deep = True)
    col_names_index_df.drop_duplicates(subset=("col_index"),inplace = True)
    salma.rename(columns=dict(zip(col_names_index_df["col_index"], col_names_index_df["COMPONENT_ATTRIB"])), inplace = True)
    
    # Stupid solution to the SAH doc that are falsly appear as RP while they are not and not all of them
    salma.dropna(subset = ['SAP Curing Code'], inplace = True)
    
    salma.to_excel('All_spec_local.xlsx',index=False)

# Checking 
Using the History approach 